In [280]:
import argparse
import io

from IPython.core.display import display, HTML
import IPython.display as ipd


from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types

from pydub import AudioSegment

import os
from scipy.io import wavfile
from scipy.fftpack import fft
import librosa
import re


import alignment_new

from pymongo import MongoClient

import json

 
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/nikitos/HeedBook-59df779e8e5a.json'

client = speech.SpeechClient()


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import yaml
import time
import glob
import sys
from datetime import datetime, timedelta
import pymorphy2
from collections import Counter

from tqdm import tqdm_notebook


In [281]:
def play_mp3(audio):
    audio.export('/Users/nikitos/export/current.wav', format='wav')
    audio, rt = librosa.load('/Users/nikitos/export/current.wav')
    display(ipd.Audio(audio, rate=rt))

In [282]:
def export_to_raw(audio):
    audio.export('/Users/nikitos/export/to_raw.wav', 'wav')
    os.system('sox -v 0.99 /Users/nikitos/export/to_raw.wav -b 16 ./input.raw channels 1 rate 16000')
    return './input.raw'

In [283]:
morph = pymorphy2.MorphAnalyzer()
sns.set() 
    
def normalize_word(word):
    # todo: hardcode
    for i in range(10):
        prev_word = word
        word = morph.parse(word)[0].normal_form
        if word == prev_word:
            return word
    return word

    # text goes to normal form
def normalize_text(text):
    return ' '.join([normalize_word(word) for word in text.split(' ')])


In [284]:
def clear_text(text):        
        text = re.sub(r'[^\w-]+', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'\s+$', '', text)
        text = text.lower()
        text = text.replace('ё', 'е')
        return text

In [285]:
def transcribe_file(speech_file):
    
    out = []
    
    client = speech.SpeechClient()

    with io.open(speech_file, 'rb') as audio_file:
        content = audio_file.read()

    audio = types.RecognitionAudio(content=content)
    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code='ru-RU',
        enable_word_time_offsets=True)
    
    response = client.recognize(config, audio)
    transcript = ''
    for result in response.results:
        alternative = result.alternatives[0]
        transcript = alternative.transcript
        #print('Transcript: {}'.format(alternative.transcript))

        for word_info in alternative.words:
            curr_word = {}
            word = word_info.word
            curr_word['Word'] = word
            start_time = word_info.start_time
            end_time = word_info.end_time
            curr_word['Begin'] = start_time.seconds + start_time.nanos * 1e-9
            curr_word['End'] = end_time.seconds + end_time.nanos * 1e-9
            out.append(curr_word)
            answ = 'Word: {}, start_time: {}, end_time: {}'.format(
                word,
                start_time.seconds + start_time.nanos * 1e-9,
                end_time.seconds + end_time.nanos * 1e-9)
            
            #print(answ)
    return response, out, transcript

In [286]:
def making_data_list(align1, align2, curr_list, path):
    data_list = []
    skip_index = 0
    for i in range(len(align1)):
        real_word = align1[i]
        recogn_word = align2[i]
        if ('#' in real_word) and ('#' in recogn_word):
            skip_index += 1
            continue
        elif ('#' in real_word):
            continue
        elif ('#' in recogn_word):
            skip_index += 1
            continue
        #print(real_word, recogn_word)
        if real_word == recogn_word:
            sample = {}
            sample['text_word'] = real_word
            sample['regonized_word'] = recogn_word
            sample['start'] = curr_list[i - skip_index]['Begin']
            sample['end'] = curr_list[i - skip_index]['End']
            sample['confidance'] = 1
            sample['file'] = path
            data_list.append(sample)
        elif real_word != recogn_word:
            sample = {}
            sample['text_word'] = real_word
            sample['regonized_word'] = recogn_word
            sample['start'] = curr_list[i - skip_index]['Begin']
            sample['end'] = curr_list[i - skip_index]['End']
            sample['confidance'] = 0
            sample['file'] = path
            data_list.append(sample)
    return data_list

In [306]:
def search_text_frame(transcript, ORIGINAL_TEXT, NORM_TEXT, words_df):
    phrase_list = transcript.split()
    size_phrase = min(5, len(phrase_list))

    N = size_phrase + 5
    PERCENT = 0.6
    
    text_list = ORIGINAL_TEXT.split(' ')

    TABOO_PHRASE = (' ').join(phrase_list[:size_phrase])
    print("Start searching in search_text_frame")
    pack = word_search(N, PERCENT, TABOO_PHRASE, ORIGINAL_TEXT, NORM_TEXT, words_df)
    print("Start alignment")
    t_s = len(text_list)
    print(t_s, pack[0][1])
    for_alignment = text_list[pack[0][1]:min(pack[0][1] + len(phrase_list) + 2, t_s - 3)]
    return for_alignment

In [313]:
def data_from_audiobooks(audio_path, text_path):
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/nikitos/HeedBook-59df779e8e5a.json'
    client = speech.SpeechClient()
    audio = AudioSegment.from_mp3(audio_path)
    
  
    ORIGINAL_TEXT = open(text_path, encoding='windows-1251').read()
    ORIGINAL_TEXT = clear_text(ORIGINAL_TEXT)
    text_list = ORIGINAL_TEXT.split()
    
    text = normalize_text(ORIGINAL_TEXT)
    words = text.split(' ')
    words_df = pd.DataFrame(words, columns=['word']).reset_index()
    print('df for text was made')
    
    total_mls = len(audio)
    step = 30000
    n_steps = total_mls // step

    data = []
    for i in tqdm_notebook(range(1, n_steps + 1)):

        start_fr = step * i

        end_fr = step * (i + 1)
        x, curr_list, transcript = transcribe_file(export_to_raw(audio[start_fr:end_fr]))
        print('part', i, 'was transcribed')
        print(transcript)
        transcript = clear_text(transcript)
        phrase_list = transcript.split()
        for_alignment = search_text_frame(transcript, ORIGINAL_TEXT, text, words_df)
        print("Text_frame was found")
        align1, align2 = alignment_new.needle(for_alignment, phrase_list)
        
        list_for_frame = making_data_list(align1, align2, curr_list, audio_path)
        print(list_for_frame)
        data += list_for_frame
        
    '''start_fr = step * number
    end_fr = total_mls - 1
    x, curr_list, transcript = transcribe_file(export_to_raw(audio[start_fr:end_fr]))
    transcript = clear_text(transcript)
    phrase_list = transcript.split()
    for_alignment = search_text_frame(transcript, ORIGINAL_TEXT, text_list, words_df)
        
    align1, align2 = alignment_new.needle(for_alignment, phrase_list)
        
    list_for_frame = making_data_list(align1, align2, curr_list)
    print(list_for_frame)
    data += list_for_frame'''
    
    
    return data

In [308]:
def word_search(N, PERCENT, TABOO_PHRASE, ORIGINAL_TEXT, text, words_df):
    
    print("searching ", TABOO_PHRASE)
    # removig some symbols 
    def clear_text(text):
        # remove non-word symbols
        text = re.sub(r'[^\w-]+', ' ', text)
        # remove extra spaces
        text = re.sub(r'\s+', ' ', text)
        # remove space at the and of the text
        text = re.sub(r'\s+$', '', text)
        text = text.lower()
        text = text.replace('ё', 'е')
        return text
    
    # word goes to normal form
    def normalize_word(word):
        # todo: hardcode
        for i in range(10):
            prev_word = word
            word = morph.parse(word)[0].normal_form
            if word == prev_word:
                return word
        return word

    # text goes to normal form
    def normalize_text(text):
        return ' '.join([normalize_word(word) for word in text.split(' ')])

    # clean and normalize text
    def prepare_text(text):
        text = clear_text(text)
        text = normalize_text(text)
        # remove in-word non-letter symbols
        text = re.sub(r'[^\w\s]', '', text)
        return text
    
    # generation list of synonyms
    def collect_synonyms(words):
        words = [prepare_text(word) for word in words]
        synonyms = []
        synonyms += words
        synonyms = list(set(synonyms))
        return synonyms    
    
    
    # generation list of synonyms
    def find_taboo(words, taboo_synonyms_df):
#         words: taboo words to find
#         taboo_synonyms_df: found synonyms in text. pd.DataFrame(columns=['word']) with index, corresponding to list of <words> from the text

#         return: a list of values of found words: [[<word>, <index>, <synonym or original word found in text>], ...]
#         example: [[['a', 0, 'a']], [['b', 2, 'b']], [['c', 3, 'd']]]
        words = list(words)
        taboo_synonyms_df = pd.DataFrame(taboo_synonyms_df)

        res_values = []

        for word in list(words):
            word_indices = taboo_synonyms_df[taboo_synonyms_df['word'] == word].index

            if len(word_indices) == 0:
                # word not found
                continue

            # remove word from found_left dataframe
            res_values.append([word] + taboo_synonyms_df.loc[word_indices[0]].tolist())
            words.remove(word)
            taboo_synonyms_df = taboo_synonyms_df.drop(word_indices[0])

        while True:
            cur_synonyms = collect_synonyms(words)
            taboo_synonyms_df = taboo_synonyms_df[taboo_synonyms_df['word'].isin(cur_synonyms)]

            if len(taboo_synonyms_df) == 0:
                # no more synonyms found in text
                break

            pairs = []

            # calc how many synonym connections with words is there
            for word in words:
                word_synonyms = collect_synonyms([word])
                for synonym in word_synonyms:
                    if synonym in taboo_synonyms_df['word'].values:
                        pairs.append([word, synonym])

            df =  pd.DataFrame(pairs, columns=['word', 'synonym'])
            df['one'] = 1

            df['synonym_power'] = df.groupby('synonym').one.transform('sum')
            df['word_power'] = df.groupby('word').one.transform('sum')

            df = df.sort_values(['synonym_power', 'word_power'])

            word, synonym = df[['word', 'synonym']].iloc[0].tolist()

            synonym_indices = taboo_synonyms_df[taboo_synonyms_df['word'] == synonym].index

            # remove word from taboo_synonyms_df dataframe
            res_values.append([word] + taboo_synonyms_df.loc[synonym_indices[0]].tolist())
            words.remove(word)
            taboo_synonyms_df = taboo_synonyms_df.drop(synonym_indices[0])
        return res_values

    
    #original_text = ORIGINAL_TEXT
    #text = NORM_TE
    #words = text_list
    
    original_text = clear_text(ORIGINAL_TEXT)
    ORIGINAL_TEXT = original_text
    #text = normalize_text(original_text)
    words = text.split(' ')
    #words_df = pd.DataFrame(words, columns=['word']).reset_index()
    taboo_phrase = normalize_text(TABOO_PHRASE)
    taboo_words = taboo_phrase.split(' ')

    taboo_synonyms = taboo_words
        
    taboo_index = [i for i, word in enumerate(words) if word in taboo_synonyms]
    taboo_words_df = words_df.iloc[taboo_index, :]

    start_time = time.time()
    cond1 = taboo_words_df['index'] - taboo_words_df['index'].shift(1).fillna(0) < N
    cond2 = taboo_words_df['index'].shift(-1).fillna(0) - taboo_words_df['index'] < N
    taboo_words_df = taboo_words_df[cond1 | cond2]
    res = []
    for i in range(taboo_words_df.shape[0]):
        index_low = taboo_words_df.index[i]
        index_high = index_low + N
        indices_in = [i for i in taboo_words_df.index if i > index_low - 1 and i < index_high + 1]
        taboo_words_in = taboo_words_df.loc[indices_in]
        pack = find_taboo(taboo_words, taboo_words_in)
        pack = list(sorted(pack, key=lambda x: x[1]))
        if len(pack) > round(PERCENT * len(taboo_words)) - 1:
            res.append(pack) 

    
    final_pack = []

    original_words = original_text.split(' ')
    beg_last = 0
    end_last = 0
    j = 0
    cmd_colors = {'red': '\033[31m', 'yellow': '\033[33m'}
    #start_time = time.time()
    for pack in res:
        pack_synonyms = list(set([x[2] for x in pack]))
        #color=get_color(words[i] in taboo_words, words[i] in pack_synonyms)
        beg, end = pack[0][1], pack[-1][1] + 1
        real_end = beg + N
        if end - beg > N:
            beg_last = beg
            end_last = end
            continue
        
        if end == end_last:
            beg_last = beg
            continue

        '''tmp = IMPORTANT_WORDS
        for i in range(beg, real_end):
            if words[i] in taboo_words:
                index = taboo_words.index(words[i])
                tmp[index] = 0 

        s = 0
        for i in range(len(tmp)):
             s = s + tmp[i]

        if s!= 0:
            continue'''

        #print('Pack: {}'.format(j))
        return pack
        final_pack.append(pack)
        j = j + 1

        #for i in range(beg, real_end):
            #print_word(original_words[i], color=get_color(words[i] in taboo_words, words[i] in pack_synonyms))

        #print()
        #print()
        end_last = end
        beg_last = beg
        
    print('--Time--', time.time() - start_time)   
    print('')
    return final_pack, len(final_pack)
        

In [309]:
data = data_from_audiobooks('./generation_p/book/0002.mp3', './generation_pi.txt')

df for text was made


A Jupyter Widget

part 1 was transcribed
уравновесить светлое начало темном несмотря на эту блестящую разработку в возрасте 18 лет татарский с удовольствием потерял свой первый паспорт а второй получил уже на имя Владимира после этого его жизнь складывалась Самым обычным образом Он поступил в технический институт Не потому Понятное дело что любил технику его специальностью были какие-то электро плавильные печи а потому что не хотел идти в Армию
Start searching in search_text_frame
searching  уравновесить светлое начало темном несмотря
Start alignment
16317 1034
Text_frame was found
Identity = 86.364 percent
Score = 525
[{'text_word': 'уравновесить', 'regonized_word': 'уравновесить', 'start': 0.0, 'end': 0.9, 'confidance': 1, 'file': './generation_p/book/0002.mp3'}, {'text_word': 'светлое', 'regonized_word': 'светлое', 'start': 0.9, 'end': 1.5, 'confidance': 1, 'file': './generation_p/book/0002.mp3'}, {'text_word': 'начало', 'regonized_word': 'начало', 'start': 1.5, 'end': 1.7000000000000002, 'confidance

part 2 was transcribed
но в 21 год с ним случилось нечто решившего дальнейшую судьбу летом в деревне он прочитал маленький Томик Бориса Пастернака стихи которым он раньше не питал никакой склонности до такой степени потрясли его что несколько недель он не мог думать ни о чём другом а потом начал писать их Steam он навсегда запомнил ржавый каркас автобус
Start searching in search_text_frame
searching  но в 21 год с
Start alignment
16317 1043
Text_frame was found
Identity = 7.937 percent
Score = -240
[{'text_word': 'в', 'regonized_word': 'в', 'start': 0.8, 'end': 1.0, 'confidance': 1, 'file': './generation_p/book/0002.mp3'}, {'text_word': 'лет', 'regonized_word': '21', 'start': 1.0, 'end': 1.8, 'confidance': 0, 'file': './generation_p/book/0002.mp3'}, {'text_word': 'татарский', 'regonized_word': 'год', 'start': 1.8, 'end': 2.3, 'confidance': 0, 'file': './generation_p/book/0002.mp3'}, {'text_word': 'с', 'regonized_word': 'с', 'start': 2.3, 'end': 3.4, 'confidance': 1, 'file': './generati

part 4 was transcribed
и он не прошёл пришлось довольствоваться переводом из языков народов СССР представлял себе свое будущее примерно так днём пустая аудитория в литинституте подстрочник с узбекского или киргизского которые нужно зарифмовать очередной дате А по вечерам труды для вечности потом незаметно произошло одно существенное для его будущего события
Start searching in search_text_frame
searching  и он не прошел пришлось
Start alignment
16317 1204
Text_frame was found
Identity = 84.000 percent
Score = 380
[{'text_word': 'он', 'regonized_word': 'он', 'start': 0.0, 'end': 0.30000000000000004, 'confidance': 1, 'file': './generation_p/book/0002.mp3'}, {'text_word': 'не', 'regonized_word': 'не', 'start': 0.30000000000000004, 'end': 0.4, 'confidance': 1, 'file': './generation_p/book/0002.mp3'}, {'text_word': 'прошел', 'regonized_word': 'прошел', 'start': 0.4, 'end': 1.1, 'confidance': 1, 'file': './generation_p/book/0002.mp3'}, {'text_word': 'пришлось', 'regonized_word': 'пришлось', '

...

part 10 was transcribed
который тоже делали вид что ничего не происходит можно было на некоторое время в это поверить конец наступил неожиданно Однажды во время прогулки татарские остановился у закрытого на обед обувного магазина zoeva витрины уплывала в летнем зное толстая миловидная продавщицы которую татарски почему-то сразу назвал про себя маленькой отреагировал разноцветных турецких подделок стояла пара
Start searching in search_text_frame
searching  который тоже делали вид что
Start alignment
16317 1500
Text_frame was found
Identity = 45.833 percent
Score = 135
[{'text_word': 'которые', 'regonized_word': 'который', 'start': 0.0, 'end': 0.4, 'confidance': 0, 'file': './generation_p/book/0002.mp3'}, {'text_word': 'тоже', 'regonized_word': 'тоже', 'start': 0.4, 'end': 0.8, 'confidance': 1, 'file': './generation_p/book/0002.mp3'}, {'text_word': 'делали', 'regonized_word': 'делали', 'start': 0.8, 'end': 1.1, 'confidance': 1, 'file': './generation_p/book/0002.mp3'}, {'text_word': 'вид'

part 12 was transcribed
пьяненький преподаватель Советской литературы и злит института называл наш Гештальт это было так жалко смешное трогательно на пряжке арфы что татарского на глаза навернулись слезы на ботинках лежал густой слой пыли они были явно не востребованы эпохой татарки знал что тоже не востребован эпохой но успел жить с этим знанием еда
Start searching in search_text_frame
searching  пьяненький преподаватель советской литературы и
Start alignment
16317 1624
Text_frame was found
Identity = 74.545 percent
Score = 340
[{'text_word': 'пьяненький', 'regonized_word': 'пьяненький', 'start': 0.0, 'end': 0.9, 'confidance': 1, 'file': './generation_p/book/0002.mp3'}, {'text_word': 'преподаватель', 'regonized_word': 'преподаватель', 'start': 0.9, 'end': 1.5, 'confidance': 1, 'file': './generation_p/book/0002.mp3'}, {'text_word': 'советской', 'regonized_word': 'советской', 'start': 1.5, 'end': 1.9, 'confidance': 1, 'file': './generation_p/book/0002.mp3'}, {'text_word': 'литературы', 

part 14 was transcribed
теплицы под этим небом не как сосут драгоценным винам А именно как ботинки с пряжками арфами Кроме того он понял еще одна вечность которой он раньше верил могла существовать только на государственных дотаций или что то же самое как нечто запрещены государством больше того существовать она могла только в качестве полу осознанно
Start searching in search_text_frame
searching  теплицы под этим небом не
Start alignment
16317 1736
Text_frame was found
Identity = 73.214 percent
Score = 335
[{'text_word': 'под', 'regonized_word': 'под', 'start': 1.1, 'end': 1.3, 'confidance': 1, 'file': './generation_p/book/0002.mp3'}, {'text_word': 'этим', 'regonized_word': 'этим', 'start': 1.3, 'end': 1.5, 'confidance': 1, 'file': './generation_p/book/0002.mp3'}, {'text_word': 'небом', 'regonized_word': 'небом', 'start': 1.5, 'end': 1.7000000000000002, 'confidance': 1, 'file': './generation_p/book/0002.mp3'}, {'text_word': 'не', 'regonized_word': 'не', 'start': 1.7000000000000002, 'e

part 16 was transcribed
это было неважно потомучто татарский Ясно понимал при любом раскладе манки просто не до вечности и когда она окончательно перестанет в неё верить никакой вечности больше не будет потому что где и тогда быть или как он записал свою книжечку Придя домой когда исчезает субъекты в вечности то исчезают и все объекты а единственным субъектом вечности является
Start searching in search_text_frame
searching  это было неважно потомучто татарский
Start alignment
16317 1295
Text_frame was found
Identity = 7.812 percent
Score = -245
[{'text_word': 'это', 'regonized_word': 'это', 'start': 0.0, 'end': 0.5, 'confidance': 1, 'file': './generation_p/book/0002.mp3'}, {'text_word': 'удар', 'regonized_word': 'было', 'start': 0.5, 'end': 0.7000000000000001, 'confidance': 0, 'file': './generation_p/book/0002.mp3'}, {'text_word': 'но', 'regonized_word': 'неважно', 'start': 0.7000000000000001, 'end': 1.0, 'confidance': 0, 'file': './generation_p/book/0002.mp3'}, {'text_word': 'его', 'r

part 18 was transcribed
стихотворение Так что такое вечность это банька вечность это банька с пауками Если эту баньку позабудет Манька что же будет с Родиной и с нами
Start searching in search_text_frame
searching  стихотворение так что такое вечность
Start alignment
16317 1382
Text_frame was found
Identity = 10.345 percent
Score = -100
[{'text_word': 'совсем', 'regonized_word': 'стихотворение', 'start': 0.0, 'end': 0.5, 'confidance': 0, 'file': './generation_p/book/0002.mp3'}, {'text_word': 'так', 'regonized_word': 'так', 'start': 0.5, 'end': 1.5, 'confidance': 1, 'file': './generation_p/book/0002.mp3'}, {'text_word': 'что', 'regonized_word': 'что', 'start': 1.5, 'end': 2.6, 'confidance': 1, 'file': './generation_p/book/0002.mp3'}, {'text_word': 'вечность', 'regonized_word': 'вечность', 'start': 3.0, 'end': 3.7, 'confidance': 1, 'file': './generation_p/book/0002.mp3'}, {'text_word': 'существовала', 'regonized_word': 'банька', 'start': 4.2, 'end': 4.4, 'confidance': 0, 'file': './gener

In [312]:
len(data)

862

In [315]:
df_data = pd.DataFrame(data)

In [317]:
df_data.confidance.value_counts()

1    599
0    263
Name: confidance, dtype: int64

In [318]:
df_data[df_data.confidance == 0]

,confidance,end,file,regonized_word,start,text_word
3,0,3.0,./generation_p/book/0002.mp3,темном,1.7,темным
11,0,7.7,./generation_p/book/0002.mp3,18,7.1,восемнадцати
50,0,27.1,./generation_p/book/0002.mp3,плавильные,26.6,электроплавильные
61,0,1.8,./generation_p/book/0002.mp3,21,1.0,лет
62,0,2.3,./generation_p/book/0002.mp3,год,1.8,татарский
64,0,3.6,./generation_p/book/0002.mp3,ним,3.4,свой
65,0,4.1,./generation_p/book/0002.mp3,случилось,3.6,первый
66,0,5.1,./generation_p/book/0002.mp3,нечто,4.1,паспорт
67,0,7.0,./generation_p/book/0002.mp3,решившего,5.1,а
68,0,7.7,./generation_p/book/0002.mp3,дальнейшую,7.0,второй


In [179]:
# max length of phrase
N = 10 

# percent of words must be in text
PERCENT = 0.6  

# the taboo phrase


# the original text
ORIGINAL_TEXT = open('/Users/nikitos/Documents/HeedBook_dataset_from_audiobooks/generation_pi.txt', encoding='windows-1251').read()

ORIGINAL_TEXT_au = clear_text(ORIGINAL_TEXT)
    
text_list = ORIGINAL_TEXT_au.split()
    
text = normalize_text(ORIGINAL_TEXT_au)
words = text.split(' ')
words_df = pd.DataFrame(words, columns=['word']).reset_index()
print('df for text was made')

df for text was made


In [271]:
ORIGINAL_TEXT_au

'generation п виктор олегович пелевин главный герой романа представитель поколения п с соответствующими юношескими идеалами опускается до торговца в киоске потом осваивает интеллектуальную халтуру на ниве рекламы а в итоге становится земным воплощением мужа богини иштар только вместо супружеской функции исполняет рекламную вся прелесть пелевинского романа в том что его каждый воспринимает по-своему это и глубокая эзотерика и блестящее надругательство над рекламой и политический памфлет и философская фантастика виктор пелевин generation п памяти среднего класса все упоминаемые в тексте торговые марки являются собственностью их уважаемых владельцев и все права сохранены названия товаров и имена политиков не указывают на реально существующие рыночные продукты и относятся только к проекциям элементов торгово-политического информационного пространства принудительно индуцированным в качестве объектов индивидуального ума автор просит воспринимать их исключительно в этом качестве остальные сов

In [180]:
start_time = time.time()
ORIGINAL_TEXT = open('/Users/nikitos/Documents/HeedBook_dataset_from_audiobooks/generation_pi.txt', encoding='windows-1251').read()

ORIGINAL_TEXT_au = clear_text(ORIGINAL_TEXT)
text_list = ORIGINAL_TEXT_au.split()
    
text = normalize_text(ORIGINAL_TEXT_au)
print('--Time--', time.time() - start_time)   
print('')

--Time-- 9.755423784255981



In [181]:
words_df = pd.DataFrame(words, columns=['word']).reset_index()

In [291]:
TABOO_PHRASE = 'воспоминания какой-нибудь маленький и забавного' 

In [302]:
# max length of phrase
N = 10

# percent of words must be in text
PERCENT = 0.6

# the taboo phrase



In [303]:
pack = word_search(N, PERCENT, TABOO_PHRASE, ORIGINAL_TEXT, text, words_df)

searching  воспоминания какой-нибудь маленький и забавного


In [304]:
pack

[['какой-нибудь', 5760, 'какой-нибудь'],
 ['маленький', 5761, 'маленький'],
 ['и', 5769, 'и']]

In [305]:
pack[0][1]

5760